In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163014 sha256=cdd484bafabbd64e706c3260e7922eeb84b6317d13472ee39ec79ec51b415713
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate


In [3]:
df = pd.read_csv('/content/dataset.csv')

<ipython-input-3-350c1d7fc2f5>:1: DtypeWarning: Columns (0,1,2,3,4,5,6,13,14,15,17,18,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/dataset.csv')


In [4]:
df.head()

,Date,Timestamp,Invoice No.,Payment Type,Order Type,Area,Item Name,Price,Qty.,Sub Total,...,Address,GST,Assign To,Non Taxable,SGST Rate,SGST Amount,CGST Rate,CGST Amount,Service Charge Rate,Service Charge Amount
0,2024-01-28,2024-01-29 02:25:22,21573,Cash,Dine In,Dining,ORANGE JUICE,285.71,1.0,285.71,...,NaN,NaN,NaN,0.0,2.5,3.52,2.5,3.52,10.0,14.07
1,2024-01-28,2024-01-29 02:17:11,21572,CARD,Dine In,Dining,Iced Latte (350 ML),214.29,1.0,214.29,...,NaN,NaN,NaN,0.0,2.5,5.36,2.5,5.36,10.0,21.43
2,2024-01-28,2024-01-29 02:17:11,21572,CARD,Dine In,Dining,ALMOND MILK (200 ML),85.71,1.0,85.71,...,NaN,NaN,NaN,0.0,2.5,2.14,2.5,2.14,10.0,8.57
3,2024-01-28,2024-01-29 02:11:30,21571,CARD,Dine In,Dining,Chicken Calzone,66.67,1.0,66.67,...,NaN,NaN,Allahbaksh Shaikh,0.0,2.5,1.67,2.5,1.67,10.0,6.67
4,2024-01-28,2024-01-29 02:11:30,21571,CARD,Dine In,Dining,Coconut Nankhatai (with Egg),285.71,1.0,285.71,...,NaN,NaN,Allahbaksh Shaikh,0.0,2.5,7.14,2.5,7.14,10.0,28.57


In [5]:
df3 = df[['Item Name', 'Price', 'Qty.', 'Sub Total', 'Discount', 'Tax', 'Final Total', 'Category']]
df3.head()

,Item Name,Price,Qty.,Sub Total,Discount,Tax,Final Total,Category
0,ORANGE JUICE,285.71,1.0,285.71,145.0,21.11,161.82,JUICES
1,Iced Latte (350 ML),214.29,1.0,214.29,0.0,32.15,246.44,Cold Coffee
2,ALMOND MILK (200 ML),85.71,1.0,85.71,0.0,12.85,98.56,Milk
3,Chicken Calzone,66.67,1.0,66.67,0.0,10.01,76.68,Food Menu
4,Coconut Nankhatai (with Egg),285.71,1.0,285.71,0.0,42.85,328.56,Sweet


In [6]:
from sklearn.preprocessing import MinMaxScaler
df3 = df[['Item Name', 'Price', 'Qty.', 'Sub Total', 'Discount', 'Tax', 'Final Total', 'Category']]

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df3[['Qty.', 'Sub Total', 'Discount', 'Tax']])

weights = {'Qty.': 0.4, 'Sub Total': 0.3, 'Discount': 0.2, 'Tax': 0.1}

weighted_scores = (normalized_data * list(weights.values())).sum(axis=1)
df3['Weighted Score'] = weighted_scores

recommended_items = df3.sort_values(by='Weighted Score', ascending=False).head(5)

print("Top 5 recommended items:")
print(recommended_items[['Item Name', 'Weighted Score']])

Top 5 recommended items:
                                  Item Name  Weighted Score
19    Origanal South Indian Frappe (350 ML)        0.700000
140      South Indian Filter Kaapi (150 ML)        0.489980
5368    Sea Salt Dark Mocha Frappe (450 ML)        0.238473
3524   Madagascar Chocochip Frappe (350 ML)        0.191497
1828                    Vietnamese (350 ML)        0.189327


<ipython-input-6-2e75c406d377>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Weighted Score'] = weighted_scores


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
df3.dropna(inplace=True)
scaler = StandardScaler()
standardized_data = scaler.fit_transform(df3[['Price', 'Qty.', 'Sub Total', 'Discount', 'Tax', 'Final Total']])
item_sim_matrix = cosine_similarity(standardized_data)
def get_recommendations(item_name, n=5):
    # Find the index of the given item
    item_index = df3.index[df3['Item Name'] == item_name].tolist()
    if not item_index:
        print("Item not found in the dataset.")
        return []
    item_sim_scores = list(enumerate(item_sim_matrix[item_index[0]]))
    item_sim_scores = sorted(item_sim_scores, key=lambda x: x[1], reverse=True)
    sim_item_indices = [i for i, _ in item_sim_scores[1:n+1]]
    recommendations = df3['Item Name'].iloc[sim_item_indices].tolist()

    return recommendations

item_name = 'Aeropress'  # Example item name
recommendations = get_recommendations(item_name)
print("Top 5 recommendations for item", item_name, ":", recommendations)

<ipython-input-8-a79d68229120>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.dropna(inplace=True)


Top 5 recommendations for item Aeropress : ['Cappucino (350 ML)', 'Pour Over', 'Cappucino (350 ML)', 'Cappucino (350 ML)', 'Iced Latte (450 ML)']
